In [1]:
import numpy as np 
import pandas as pd 
import re
import os
import glob
import pickle
from sklearn.model_selection import train_test_split 
import librosa

In [52]:
from keras.models import Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import Activation
from keras.layers import concatenate
from keras.layers import Reshape
from keras import optimizers
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import tensorflow as tf
from keras.layers.recurrent import LSTM


In [3]:
INPUT_FOLDER = "input/"
# INPUT_FOLDER = "../input/"
print(os.listdir(INPUT_FOLDER))

['train_curated.csv', 'train_noisy.csv', 'sample_submission.csv', 'train_noisy', 'train_curated', 'test']


In [4]:
TRAIN_CURATED_PATH = INPUT_FOLDER + "train_curated.csv"
TRAIN_NOISY_PATH = INPUT_FOLDER + "train_noisy.csv"
SAMPLE_SUBMISSION_PATH = INPUT_FOLDER + "sample_submission.csv"
TRAIN_CURATED = INPUT_FOLDER + "train_curated/"
TRAIN_NOISY = INPUT_FOLDER + "train_noisy/"
TEST = INPUT_FOLDER + "test/"

train_curated = pd.read_csv(TRAIN_CURATED_PATH)
train_noisy = pd.read_csv(TRAIN_NOISY_PATH)
sample = pd.read_csv(SAMPLE_SUBMISSION_PATH)

In [7]:
target_names = sample.columns[1:]
num_targets = len(target_names)

src_dict = {target_names[i]:i for i in range(num_targets)}
src_dict_inv = {i:target_names[i] for i in range(num_targets)}

In [8]:
num_freq = 128
len_div = 256

In [76]:
n_hidden = 600
n_hidden1 = 300

inputs = Input(shape=(num_freq,len_div), name='input')

lstm = LSTM(n_hidden, batch_input_shape=(None, num_freq, len_div), return_sequences=False, name='LSTM')(inputs)
reshape = Reshape((n_hidden, 1))(lstm)
norm = BatchNormalization(axis=-1, name='norm')(reshape)
lstm1 = LSTM(n_hidden1, batch_input_shape=(None, n_hidden,1), return_sequences=False, name='LSTM1')(norm)
drop1 = Dropout(rate=0.05)(lstm1)
# flatten1 = Flatten(name='flatten1')(drop1)
dense1 = Dense(80, name='dense1')(drop1)
pred = Activation('softmax',name='pred')(dense1)

adam = optimizers.Adam(lr=0.01)

model = Model(inputs=inputs, outputs=pred)

In [77]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 128, 256)          0         
_________________________________________________________________
LSTM (LSTM)                  (None, 600)               2056800   
_________________________________________________________________
reshape_9 (Reshape)          (None, 600, 1)            0         
_________________________________________________________________
norm (BatchNormalization)    (None, 600, 1)            4         
_________________________________________________________________
LSTM1 (LSTM)                 (None, 300)               362400    
_________________________________________________________________
dropout_14 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense1 (Dense)               (None, 80)                24080     
__________

In [21]:
def tf_one_sample_positive_class_precisions(y_true, y_pred) :
    num_samples, num_classes = y_pred.shape
    
    # find true labels
    pos_class_indices = tf.where(y_true > 0) 
    
    # put rank on each element
    retrieved_classes = tf.nn.top_k(y_pred, k=num_classes).indices
    sample_range = tf.zeros(shape=tf.shape(tf.transpose(y_pred)), dtype=tf.int32)
    sample_range = tf.add(sample_range, tf.range(tf.shape(y_pred)[0], delta=1))
    sample_range = tf.transpose(sample_range)
    sample_range = tf.reshape(sample_range, (-1,num_classes*tf.shape(y_pred)[0]))
    retrieved_classes = tf.reshape(retrieved_classes, (-1,num_classes*tf.shape(y_pred)[0]))
    retrieved_class_map = tf.concat((sample_range, retrieved_classes), axis=0)
    retrieved_class_map = tf.transpose(retrieved_class_map)
    retrieved_class_map = tf.reshape(retrieved_class_map, (tf.shape(y_pred)[0], num_classes, 2))
    
    class_range = tf.zeros(shape=tf.shape(y_pred), dtype=tf.int32)
    class_range = tf.add(class_range, tf.range(num_classes, delta=1))
    
    class_rankings = tf.scatter_nd(retrieved_class_map,
                                          class_range,
                                          tf.shape(y_pred))
    
    #pick_up ranks
    num_correct_until_correct = tf.gather_nd(class_rankings, pos_class_indices)

    # add one for division for "presicion_at_hits"
    num_correct_until_correct_one = tf.add(num_correct_until_correct, 1) 
    num_correct_until_correct_one = tf.cast(num_correct_until_correct_one, tf.float32)
    
    # generate tensor [num_sample, predict_rank], 
    # top-N predicted elements have flag, N is the number of positive for each sample.
    sample_label = pos_class_indices[:, 0]   
    sample_label = tf.reshape(sample_label, (-1, 1))
    sample_label = tf.cast(sample_label, tf.int32)
    
    num_correct_until_correct = tf.reshape(num_correct_until_correct, (-1, 1))
    retrieved_class_true_position = tf.concat((sample_label, 
                                               num_correct_until_correct), axis=1)
    retrieved_pos = tf.ones(shape=tf.shape(retrieved_class_true_position)[0], dtype=tf.int32)
    retrieved_class_true = tf.scatter_nd(retrieved_class_true_position, 
                                         retrieved_pos, 
                                         tf.shape(y_pred))
    # cumulate predict_rank
    retrieved_cumulative_hits = tf.cumsum(retrieved_class_true, axis=1)

    # find positive position
    pos_ret_indices = tf.where(retrieved_class_true > 0)

    # find cumulative hits
    correct_rank = tf.gather_nd(retrieved_cumulative_hits, pos_ret_indices)  
    correct_rank = tf.cast(correct_rank, tf.float32)

    # compute presicion
    precision_at_hits = tf.truediv(correct_rank, num_correct_until_correct_one)

    return pos_class_indices, precision_at_hits

def tf_lwlrap(y_true, y_pred):
    num_samples, num_classes = y_pred.shape
    pos_class_indices, precision_at_hits = (tf_one_sample_positive_class_precisions(y_true, y_pred))
    pos_flgs = tf.cast(y_true > 0, tf.int32)
    labels_per_class = tf.reduce_sum(pos_flgs, axis=0)
    weight_per_class = tf.truediv(tf.cast(labels_per_class, tf.float32),
                                  tf.cast(tf.reduce_sum(labels_per_class), tf.float32))
    sum_precisions_by_classes = tf.zeros(shape=(num_classes), dtype=tf.float32)  
    class_label = pos_class_indices[:,1]
    sum_precisions_by_classes = tf.unsorted_segment_sum(precision_at_hits,
                                                        class_label,
                                                       num_classes)
    labels_per_class = tf.cast(labels_per_class, tf.float32)
    labels_per_class = tf.add(labels_per_class, 1e-7)
    per_class_lwlrap = tf.truediv(sum_precisions_by_classes,
                                  tf.cast(labels_per_class, tf.float32))
    out = tf.cast(tf.tensordot(per_class_lwlrap, weight_per_class, axes=1), dtype=tf.float32)
    return out

In [78]:
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=[tf_lwlrap])

In [29]:
with open('preprocessed_dataset/train_arr_0.pickle', 'rb') as f:
    X_train0 = pickle.load(f)
    y_train0 = pickle.load(f)
with open('preprocessed_dataset/train_arr_1.pickle', 'rb') as f:
    X_train1 = pickle.load(f)
    y_train1 = pickle.load(f)
    
X_train = np.vstack([X_train0, X_train1])
y_train = np.vstack([y_train0, y_train1])

X_train = X_train.reshape(-1, num_freq,len_div)

In [35]:
with open('preprocessed_dataset/val_arr_0.pickle', 'rb') as f:
    X_val0 = pickle.load(f)
    y_val0 = pickle.load(f)
with open('preprocessed_dataset/val_arr_1.pickle', 'rb') as f:
    X_val1 = pickle.load(f)
    y_val1 = pickle.load(f)
    
X_val = np.vstack([X_val0, X_val1])
y_val = np.vstack([y_val0, y_val1])
X_val = X_val.reshape(-1, num_freq,len_div)

In [62]:
datagen = ImageDataGenerator(
           rotation_range=0,
           width_shift_range=128,
           height_shift_range=0,
           shear_range=0,
           zoom_range=0,
           horizontal_flip=False,
           vertical_flip=False)

In [79]:
batch_size = 64
epochs = 20

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_val, y_val))

Train on 7550 samples, validate on 1827 samples
Epoch 1/20
7550/7550 [==============================] - 109s 14ms/step - loss: 5.3251 - tf_lwlrap: 0.0904 - val_loss: 5.0394 - val_tf_lwlrap: 0.0860
Epoch 2/20
7550/7550 [==============================] - 107s 14ms/step - loss: 5.1882 - tf_lwlrap: 0.0930 - val_loss: 5.0637 - val_tf_lwlrap: 0.0918
Epoch 3/20
7550/7550 [==============================] - 103s 14ms/step - loss: 5.1462 - tf_lwlrap: 0.0943 - val_loss: 4.9941 - val_tf_lwlrap: 0.0984
Epoch 4/20
7550/7550 [==============================] - 103s 14ms/step - loss: 5.0893 - tf_lwlrap: 0.1087 - val_loss: 4.9051 - val_tf_lwlrap: 0.1238
Epoch 5/20
7550/7550 [==============================] - 102s 14ms/step - loss: 5.0306 - tf_lwlrap: 0.1193 - val_loss: 4.8733 - val_tf_lwlrap: 0.1019
Epoch 6/20
7550/7550 [==============================] - 102s 14ms/step - loss: 4.9294 - tf_lwlrap: 0.1282 - val_loss: 4.7560 - val_tf_lwlrap: 0.1271
Epoch 7/20
 256/7550 [>.............................] - ET

KeyboardInterrupt: 